In [1]:

import pandas as pd
import numpy as np

In [2]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

In [3]:
from itertools import product

In [4]:
def ensemble(files):
    pred_ary = []
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    for f in files:
        df = pd.read_csv(f)
        pred = df[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
        pred_ary.append(pred)
        labels = df["answer"].map(option_to_index).values
        print(f"{f}: {map_at_3(pred, labels)}")
    rets = []
    for weights in product(*[np.arange(0, 1, 0.1)] * len(files)):
        
        pred = (pred_ary * np.array(weights).reshape(-1, 1, 1)).sum(axis=0)
        score = map_at_3(pred, labels)
        
        ret = {}
        for i in range(len(files)):
            ret[f"weight_{i}"] = weights[i]
        ret["score"] = score
        rets.append(ret)
    return pd.DataFrame(rets)

In [5]:
df_ret_test = ensemble([
    "../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/test_predictions.csv",
    "../output/stage2/exp005.py/20230926143713_new_data_all300val_maxlen256_bs2_OpenAssistant/reward-model-deberta-v3-base/test_predictions.csv",
    "../output/stage2/exp005.py/20230926162839_new_data_all300val_maxlen256_bs2_microsoft/deberta-v3-large/test_predictions.csv",
])

../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/test_predictions.csv: 0.9408333333333333
../output/stage2/exp005.py/20230926143713_new_data_all300val_maxlen256_bs2_OpenAssistant/reward-model-deberta-v3-base/test_predictions.csv: 0.92
../output/stage2/exp005.py/20230926162839_new_data_all300val_maxlen256_bs2_microsoft/deberta-v3-large/test_predictions.csv: 0.9408333333333333


In [6]:
df_ret_test.sort_values("score")

,weight_0,weight_1,weight_2,score
0,0.0,0.0,0.0,0.378333
90,0.0,0.9,0.0,0.920000
80,0.0,0.8,0.0,0.920000
70,0.0,0.7,0.0,0.920000
60,0.0,0.6,0.0,0.920000
...,...,...,...,...
956,0.9,0.5,0.6,0.952500
634,0.6,0.3,0.4,0.952500
806,0.8,0.0,0.6,0.954167
705,0.7,0.0,0.5,0.954167


In [7]:
df_ret_valid = ensemble([
    "../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/valid_predictions.csv",
    "../output/stage2/exp005.py/20230926143713_new_data_all300val_maxlen256_bs2_OpenAssistant/reward-model-deberta-v3-base/valid_predictions.csv",
    "../output/stage2/exp005.py/20230926162839_new_data_all300val_maxlen256_bs2_microsoft/deberta-v3-large/valid_predictions.csv",
])

../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/valid_predictions.csv: 0.867008547008549
../output/stage2/exp005.py/20230926143713_new_data_all300val_maxlen256_bs2_OpenAssistant/reward-model-deberta-v3-base/valid_predictions.csv: 0.8130341880341914
../output/stage2/exp005.py/20230926162839_new_data_all300val_maxlen256_bs2_microsoft/deberta-v3-large/valid_predictions.csv: 0.8658974358974374


In [8]:
df_ret_valid.sort_values("score")

,weight_0,weight_1,weight_2,score
0,0.0,0.0,0.0,0.422350
40,0.0,0.4,0.0,0.813034
50,0.0,0.5,0.0,0.813034
60,0.0,0.6,0.0,0.813034
70,0.0,0.7,0.0,0.813034
...,...,...,...,...
867,0.8,0.6,0.7,0.871325
978,0.9,0.7,0.8,0.871453
869,0.8,0.6,0.9,0.871496
868,0.8,0.6,0.8,0.871538


In [24]:
df = pd.read_pickle("20230925_ret.pickle")

,model_name,max_length,num_content,map3,predictions
0,../output/stage2/exp005.py/20230923195407_new_...,256,2,0.867009,"[[-1.337, 1.047, 8.664, -2.17, 1.1875], [-6.95..."
1,../output/stage2/exp005.py/20230923195407_new_...,384,3,0.869444,"[[-2.62, -1.216, 8.02, -3.955, -0.3167], [-6.5..."
2,../output/stage2/exp005.py/20230923195407_new_...,512,4,0.867863,"[[-1.972, 0.662, 8.08, -3.24, 0.0765], [-6.13,..."
3,../output/stage2/exp005.py/20230923195407_new_...,768,7,0.869103,"[[-0.787, 2.795, 5.69, -1.655, 1.073], [-7.145..."
4,../output/stage2/exp005.py/20230924021914_new_...,384,3,0.871111,"[[-1.577, -1.061, 8.01, -3.428, -0.8506], [-6...."
5,../output/stage2/exp005.py/20230924021914_new_...,512,4,0.869573,"[[-1.027, 0.09576, 8.16, -3.041, -0.313], [-6...."
6,../output/stage2/exp005.py/20230924021914_new_...,768,7,0.869060,"[[-0.3408, 1.863, 5.83, -2.057, 0.4429], [-7.5..."
7,../output/stage2/exp005.py/20230924084819_new_...,512,4,0.872692,"[[0.7046, 2.053, 9.43, -0.767, 1.345], [-6.453..."
8,../output/stage2/exp005.py/20230924084819_new_...,768,7,0.872179,"[[0.7993, 3.625, 7.414, -0.2031, 2.418], [-8.2..."


In [6]:
def softmax(x):
    x = np.exp(x)
    return x / x.sum(axis=1, keepdims=True)

In [8]:
pred_1 = df1[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
pred_1 = (pred_1 - pred_1.mean()) / pred_1.std()
pred_2 = df2[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
pred_2 = (pred_2 - pred_2.mean()) / pred_2.std()

0.9358333333333333

In [11]:
map_at_3(pred_2, labels)

0.935

In [12]:
for ratio in np.arange(0, 1.05, 0.05):
    print(f"ratio:{ratio} map3: {map_at_3(pred_1*ratio + pred_2*(1-ratio), labels)}")

ratio:0.0 map3: 0.935
ratio:0.05 map3: 0.9341666666666667
ratio:0.1 map3: 0.9316666666666668
ratio:0.15000000000000002 map3: 0.9341666666666667
ratio:0.2 map3: 0.9383333333333335
ratio:0.25 map3: 0.9358333333333334
ratio:0.30000000000000004 map3: 0.9366666666666668
ratio:0.35000000000000003 map3: 0.9383333333333332
ratio:0.4 map3: 0.9383333333333332
ratio:0.45 map3: 0.9408333333333333
ratio:0.5 map3: 0.9441666666666667
ratio:0.55 map3: 0.9441666666666667
ratio:0.6000000000000001 map3: 0.9441666666666667
ratio:0.65 map3: 0.9441666666666667
ratio:0.7000000000000001 map3: 0.945
ratio:0.75 map3: 0.9475
ratio:0.8 map3: 0.95
ratio:0.8500000000000001 map3: 0.9475
ratio:0.9 map3: 0.9408333333333333
ratio:0.9500000000000001 map3: 0.9383333333333332
ratio:1.0 map3: 0.9358333333333333
